In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [19]:
df = pd.read_csv(r"C:\Users\KIIT0001\Downloads\loan.csv.csv")

In [4]:
print(df.head())

    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0   

In [5]:
print(df.isnull().sum())

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [6]:
df["LoanAmount"] = df["LoanAmount"].fillna(df["LoanAmount"].median())
df["Loan_Amount_Term"] = df["Loan_Amount_Term"].fillna(df["Loan_Amount_Term"].median())
df["Credit_History"] = df["Credit_History"].fillna(df["Credit_History"].median())

categorical_fill_cols = ["Gender", "Married", "Dependents", "Self_Employed"]
for col in categorical_fill_cols:
    df[col] = df[col].fillna(df[col].mode()[0])


In [7]:
df.drop("Loan_ID", axis=1, inplace=True)


In [8]:
categorical_cols = [
    "Gender", "Married", "Dependents", 
    "Education", "Self_Employed", 
    "Property_Area", "Loan_Status"
]

encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])


In [9]:
X = df.drop("Loan_Status", axis=1)
y = df["Loan_Status"]


In [10]:
# Scale features (important for convergence)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [11]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [12]:
# Train Logistic Regression
model = LogisticRegression(max_iter=1000, solver='lbfgs')
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [13]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)


Model Accuracy: 0.7886178861788617


In [14]:
# Sample input:
# Gender, Married, Dependents, Education, Self_Employed,
# ApplicantIncome, CoapplicantIncome, LoanAmount,
# Loan_Amount_Term, Credit_History, Property_Area

sample_applicant = np.array([[1, 1, 0, 1, 0, 5000, 0, 150, 360, 1, 2]])

prediction = model.predict(sample_applicant)

if prediction[0] == 1:
    print("✅ Loan Approved")
else:
    print("❌ Loan Rejected")


❌ Loan Rejected


In [20]:
import pickle

# save trained model
pickle.dump(model, open("loan_model.pkl", "wb"))
